In [1]:
pip install sentence-transformers chromadb faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 20.7 MB/s eta

## Практическая задача: Медицинский чат-бот с семантическим поиском

**Цель:** Создать интеллектуальную систему, которая может находить релевантную медицинскую информацию в базе знаний и отвечать на вопросы пользователей.

Дописать код для создания чат-бота, который:

1. **Индексирует** медицинские документы в векторной базе
2. **Находит** наиболее релевантные документы по запросу пользователя  
3. **Формирует** осмысленные ответы на основе найденной информации



In [2]:
from sentence_transformers import SentenceTransformer
import chromadb
import faiss
import numpy as np

In [7]:
class MedicalChatBot:
    '''Медицинский "чат-бот", который ищет релевантное предложение из базы знаний'''
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
        self.client = chromadb.Client()
        self.collection = self.client.create_collection(name='med_db')
        self.collection_custom = self.client.create_collection(name='custom_embed')

    def create_faiss_index(self, documents, query):
      '''Создает faiss индекс и осуществляет поиск с помощью 3 моделей'''
      models_to_test = [
    'all-MiniLM-L6-v2',
    'all-mpnet-base-v2',
    'paraphrase-multilingual-MiniLM-L12-v2'
]
      for model_f in models_to_test:
        print(f'\nЗапускаем FAISS с моделью {model_f}')
        # создаем эмбеддинги
        model_faiss = SentenceTransformer(model_f)
        embeddings = model_faiss.encode(documents)

        # создаем faiss индекс
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatIP(dimension)

        # нормализуем эмбеддинги
        faiss.normalize_L2(embeddings)
        index.add(embeddings.astype('float32'))

        # эмбеддинг для запроса
        query_embedding = model_faiss.encode([query])
        faiss.normalize_L2(query_embedding)

        # собственно поиск
        D, I = index.search(query_embedding.astype('float32'), k=1)
        for i, (idx, score) in enumerate(zip(I[0], D[0])):
            print(f"Ответ FAISS, модель {model_f}: {documents[idx]} (релевантность: {score * 100:.2f}%)")


    def chroma_client_default(self, documents, query):
        """Поиск с помощью дефолтной модели хрома дб"""
        # TODO: Реализовать загрузку документов с созданием эмбеддингов
        # и сохранением в ChromaDB коллекцию
        self.collection.add(
            documents=documents,
            ids=[f"id{i}" for i in range(len(documents))]
        )
        results = self.collection.query(
            query_texts=query,
            n_results=1
        )

        print(f'\nОтвет ChromaDB, модель по умолчанию: {results['documents'][0][0]} (релевантность: {results['distances'][0][0] * 100:.2f}%)')

        # для практической работы - вывод 3 ответов в порядке релевантности, для дз оставил 1 ответ
        # if results['documents'][0]:
        #   length = len(self.results['documents'][0])
        #   context_documents = sorted([(text, score) for (text, score) in zip(self.results['documents'][0], self.results['distances'][0])], reverse=True, key=lambda x: x[1])

        #   print(f'\nОтвет ChromaDB, модель по умолчанию: {context_documents[0][0]} (релевантность: {context_documents[0][1] * 100:.2f}%)')
        #   if len(context_documents) > 1:
        #     print('Дополнительная информация:')
        #     for text, score in context_documents[1:]:
        #       print(f'{text} (релевантность: {score * 100:.2f}%)')
        # else:
        #   print('Нет релевантной информации')

    def chroma_client_custom(self, documents, query):
        '''Поиск с помощью кастомной (сберт) модели хрома дб'''
        model = SentenceTransformer("ai-forever/sbert_large_nlu_ru")
        embeddings = model.encode(documents).tolist()

        self.collection_custom.add(
            documents=documents,
            embeddings=embeddings,
            ids=[f"id{i}" for i in range(len(documents))]
        )

        query_embedding = model.encode([query]).tolist()

        results = self.collection_custom.query(
            query_embeddings=query_embedding,  # Ищем по нашим эмбеддингам
            n_results=1
        )

        print(f'\nОтвет ChromaDB, кастомная модель: {results['documents'][0][0]} (релевантность {results['distances'][0][0] * 100:.2f}%)')


    def chroma_persistent(self, documents, query):
        '''Поиск с помощью персистент клиента хрома дб'''
        import tempfile, os

        temp_dir = tempfile.mkdtemp()
        chroma_pers = chromadb.PersistentClient(path=temp_dir)

        collection_persistent = chroma_pers.create_collection(name='persistent')

        collection_persistent.add(
            documents=documents,
            ids=[f"id{i}" for i in range(len(documents))]
        )

        query_embedding = self.model.encode([query]).tolist()

        results = collection_persistent.query(
            query_texts=query,
            n_results=1
        )

        print(f'\nОтвет ChromaDB, PersistentClient, модель по умолчанию, директория для данных - {temp_dir}: {results['documents'][0][0]} (релевантность {results['distances'][0][0] * 100:.2f}%)')

    def chat(self, documents, user_query):
        """Основной метод для взаимодействия с ботом"""
        # TODO: Реализовать полный цикл:
        # 1. Поиск похожих документов
        # 2. Генерация ответа
        # 3. Возврат результата

        print('Запуск медицинского бота')
        print(f'ЗАПРОС: {user_query}')
        self.create_faiss_index(documents, user_query)
        self.chroma_client_default(documents, user_query)
        self.chroma_client_custom(documents, user_query)
        self.chroma_persistent(documents, user_query)
        print('\nЗапрос полностью обработан')




In [10]:
# ТЕСТОВЫЕ ЗАПРОСЫ
test_queries = [
    "Какие симптомы у диабета?",
    "Как лечить высокое давление?",
    "Что такое бронхиальная астма?",
    "Какие препараты используются при гипертензии?",
    "Как диагностируют остеохондроз?"
]

# БАЗА ЗНАНИЙ ДЛЯ ТЕСТИРОВАНИЯ - добавил еще синтетических предложений
medical_documents = [
    "Артериальная гипертензия - это стойкое повышение артериального давления выше 140/90 мм рт.ст.",
    "Диагностика гипертонии включает регулярные измерения давления, ЭКГ и анализы крови.",
    "Лечение гипертензии начинают с изменения образа жизни: снижение веса, ограничение соли, физическая активность.",
    "При неэффективности немедикаментозной терапии назначают антигипертензивные препараты: ингибиторы АПФ, бета-блокаторы, диуретики.",
    "Сахарный диабет 2 типа характеризуется инсулинорезистентностью и относительной инсулиновой недостаточностью.",
    "Диагностика диабета включает измерение уровня глюкозы крови натощак и тест на толерантность к глюкозе.",
    "Основные симптомы диабета: повышенная жажда, частое мочеиспускание, усталость, медленное заживление ран.",
    "Лечение диабета 2 типа включает диету, физические упражнения, метформин и при необходимости инсулин.",
    "Бронхиальная астма - хроническое воспалительное заболевание дыхательных путей.",
    "Симптомы астмы: одышка, свистящее дыхание, кашель, чувство стеснения в груди.",
    "Для лечения астмы используются ингаляционные кортикостероиды и бронходилататоры.",
    "Остеохондроз - дегенеративное поражение хряща и подлежащей кости.",
    "Симптомы остеохондроза: боль в пораженном отделе позвоночника, ограничение подвижности.",
    "Лечение остеохондроза включает ЛФК, физиотерапию, противовоспалительные препараты.",
    "Нормальное артериальное давление составляет менее 120/80 мм рт.ст.",
    "Гипертонический криз требует неотложной медицинской помощи при давлении выше 180/120.",
    "Факторы риска гипертонии: курение, ожирение, стресс и наследственность.",
    "Мониторинг давления в домашних условиях помогает контролировать гипертензию.",
    "Диуретики способствуют выведению избытка жидкости и снижению давления.",
    "Гликированный гемоглобин (HbA1c) отражает средний уровень глюкозы за 3 месяца.",
    "Осложнения диабета включают нефропатию, ретинопатию и нейропатию.",
    "Инсулинорезистентность означает снижение чувствительности клеток к инсулину.",
    "Гипогликемия - опасное состояние при слишком низком уровне сахара в крови.",
    "Диабетическая диета ограничивает простые углеводы и насыщенные жиры.",
    "Астматические приступы могут провоцироваться аллергенами, холодным воздухом и физической нагрузкой.",
    "Пикфлоуметрия используется для мониторинга функции легких при астме.",
    "Бронходилататоры быстрого действия применяются для купирования острых приступов.",
    "Хроническое воспаление при астме приводит к гиперреактивности бронхов.",
    "Профилактика астмы включает избегание триггеров и регулярный прием базисной терапии.",
    "Остеохондроз чаще поражает шейный и поясничный отделы позвоночника.",
    "Межпозвонковые грыжи являются частым осложнением остеохондроза.",
    "Магнитно-резонансная томография (МРТ) наиболее точно диагностирует остеохондроз.",
    "Мышечные спазмы часто сопровождают болевой синдром при остеохондрозе.",
    "Хондропротекторы могут замедлять прогрессирование дегенеративных изменений.",
    "Регулярные профилактические осмотры помогают выявить заболевания на ранних стадиях.",
    "Здоровый образ жизни - основа профилактики хронических заболеваний.",
    "Самолечение может быть опасным и должно согласовываться с врачом.",
    "Курение и злоупотребление алкоголем усугубляют течение многих заболеваний.",
    "Физическая активность улучшает состояние сердечно-сосудистой и дыхательной систем."
]

Домашнее задание:

- Добавить функцию для поиска с помощью FAISS
    - Функция возвращает результаты от 3х разных эмбеддинговых моделей
- Добавить PersistentClient к текущей реализации ChromaDB
    - Функция возвращает результат поиска и адрес временного хранилища базы данных
- Добавить любую кастомную модель к реализации ChromaDB (для Client)
    - Функция возвращает результат работы этой модели  

Образец вывода:

```
ЗАПРОС: ...
БОТ:
    - Ответ FAISS (модель 1: ...): ...
    - Ответ FAISS (модель 2: ...): ...
    - Ответ FAISS (модель 3: ...): ...
    - Ответ ChromaDB (Client, модель по умолчанию): ...
    - Ответ ChromaDB (Client, кастомная модель ...): ...
    - Ответ ChromaDB (PersistentClient, модель по умолчанию, имя временного файла: ...): ...
```

*Достаточно протестировать модель на одном любом запросе*

In [8]:
bot = MedicalChatBot()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
for query in test_queries:
  bot.chat(medical_documents, query)

Запуск медицинского бота
ЗАПРОС: Какие симптомы у диабета?

Запускаем FAISS с моделью all-MiniLM-L6-v2
Ответ FAISS, модель all-MiniLM-L6-v2: Осложнения диабета включают нефропатию, ретинопатию и нейропатию. (релевантность: 66.56%)

Запускаем FAISS с моделью all-mpnet-base-v2
Ответ FAISS, модель all-mpnet-base-v2: Диуретики способствуют выведению избытка жидкости и снижению давления. (релевантность: 78.78%)

Запускаем FAISS с моделью paraphrase-multilingual-MiniLM-L12-v2
Ответ FAISS, модель paraphrase-multilingual-MiniLM-L12-v2: Основные симптомы диабета: повышенная жажда, частое мочеиспускание, усталость, медленное заживление ран. (релевантность: 87.46%)

Ответ ChromaDB, модель по умолчанию: Осложнения диабета включают нефропатию, ретинопатию и нейропатию. (релевантность: 66.89%)

Ответ ChromaDB, кастомная модель: Основные симптомы диабета: повышенная жажда, частое мочеиспускание, усталость, медленное заживление ран. (релевантность 62.75%)

Ответ ChromaDB, PersistentClient, модель по у

In [19]:
import pandas as pd

data = pd.read_csv('malcovi_df.csv').dropna(subset='text')
texts = data['text'].tolist()

In [22]:
bot.chat(texts, 'Какие самые интересные экспонаты')

Запуск медицинского бота
ЗАПРОС: Какие самые интересные экспонаты

Запускаем FAISS с моделью all-MiniLM-L6-v2
Ответ FAISS, модель all-MiniLM-L6-v2: Интересные экспонаты.
Само здание тоже историческое.
Познавательно и красиво. (релевантность: 83.16%)

Запускаем FAISS с моделью all-mpnet-base-v2
Ответ FAISS, модель all-mpnet-base-v2: Интересная экскурсия, богатая коллекция экспонатов. Экскурсовод - супер! С любовью провела экскурсию. (релевантность: 83.92%)

Запускаем FAISS с моделью paraphrase-multilingual-MiniLM-L12-v2
Ответ FAISS, модель paraphrase-multilingual-MiniLM-L12-v2: Интересные сувениры (релевантность: 80.39%)

Ответ ChromaDB, модель по умолчанию: Интересные экспонаты.
Само здание тоже историческое.
Познавательно и красиво. (релевантность: 33.67%)

Ответ ChromaDB, кастомная модель: В музее предоставлено много видов различной посуды и предметов из хрусталя. Такое ощущение что хотелось бы увидеть что-то поинтереснее. (релевантность 66.63%)

Ответ ChromaDB, PersistentClient, мод

In [43]:
!git commit -m "Initialize deep-learning folder"
!git push origin main

[main fc1ce16] Initialize deep-learning folder
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 deep-learning/.gitkeep
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (4/4), 344 bytes | 344.00 KiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:VladLoPG/Lobanov-Programming-HSE.git
   0006a0a..fc1ce16  main -> main


In [44]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [51]:
!mv 'Лобанов_search_practice.ipynb' deep-learning
!git add deep-learning/
!git commit -m 'Adding search hw'
!git push origin main

mv: cannot stat 'Лобанов_search_practice.ipynb': No such file or directory
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!find /content/drive/MyDrive -name "*Лобанов*" -type f 2>/dev/null

/content/drive/MyDrive/Лобанов В.В./Сертификат вожатого Лобанов.pdf
/content/drive/MyDrive/Лобанов В.В./Выписка из трудовой книжки Лобанов.pdf
/content/drive/MyDrive/Лобанов В.В./Лобанов Прививка Гепатит В.pdf
/content/drive/MyDrive/Лобанов В.В./Лобанов Справка Из Вуза.pdf
/content/drive/MyDrive/Лобанов ВВ QR.pdf
/content/drive/MyDrive/Университетик/Резюме 26.04 Лобанов В. Федченко М..gdoc
/content/drive/MyDrive/Университетик/Преддипломная практика отчеты /37/Ворд документы без подписей/Производственная практика (преддипломная)_20.Б37_ЛобановВВ.docx
/content/drive/MyDrive/Университетик/Преддипломная практика отчеты /37/Пдф сканы с подписями Хомяковой и студента/Производственная практика (преддипломная)_20.Б37_ЛобановВВ.pdf
/content/drive/MyDrive/Лобанов Владислав.JPG
/content/drive/MyDrive/ЛобановВладислав.jpg
/content/drive/MyDrive/Лобанов ВВ.png
/content/drive/MyDrive/Colab Notebooks/Лобанов 1_arithmetics.ipynb
/content/drive/MyDrive/Colab Notebooks/Лобанов + Коваленко "Айтрекинг